In [1]:
%matplotlib inline
import pickle
import copy
import numpy as np
import random
import os
import torch
from test_utils import *
from ipywidgets import interact


# 데이터 직접 입력

In [2]:
import argparse
os.chdir('..')
from lib.utils.learning import *
from lib.utils.tools import *

In [3]:
from test_utils import *

h36m_connections = [
    (0,1),
    (0,4),
    (0,7),
    (1,2),
    (2,3),
    (4,5),
    (5,6),
    (7,8),
    (8,9),
    (9,10),
    (8,14),
    (14,15),
    (15,16),
    (8,11),
    (11,12),
    (12,13)
]

In [5]:
def draw_skeleton_2d(points, connections, elevation=0, azimuth=0):
    fig = go.Figure()
    
    # Add points
    for point in points.values():
        fig.add_trace(go.Scatter(x=[point[0]], y=[point[1]],
                                   mode='markers', marker=dict(size=2, color='blue')))

    # Add connections
    for connection in connections:
        p1, p2 = points[connection[0]], points[connection[1]]
        fig.add_trace(go.Scatter(x=[p1[0], p2[0]], y=[p1[1], p2[1]],
                                   mode='lines', line=dict(color='black')))

    img_width = 1920
    img_height = 1080
    scale_factor = 0.5

    # https://wikidocs.net/185955 참고
    fig.update_xaxes(range=[-1, 1])
    fig.update_yaxes(range=[1, -1])
    #fig.update_xaxes(autorange="reversed") # 축 범위 반전, 위의 range 설정이 초기화되버림

    fig.update_layout(width=700,height=700)
    
    fig.update_layout(template="plotly_white")

    fig.show()

In [30]:
def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--config", type=str, default="/home/hrai/codes/MotionBERT/configs/pose3d/MB_ft_h36m.yaml", help="Path to the config file.")
    parser.add_argument('-c', '--checkpoint', default='', type=str, metavar='PATH', help='checkpoint directory')
    parser.add_argument('-p', '--pretrained', default='', type=str, metavar='PATH', help='pretrained checkpoint directory')
    parser.add_argument('-r', '--resume', default='', type=str, metavar='FILENAME', help='checkpoint to resume (file name)')
    #parser.add_argument('-e', '--evaluate', default='/home/hrai/codes/MotionBERT/checkpoint/pose3d/FT_MB_ft_h36m_MB_ft_aihub_243_upbeat-pond-13/latest_epoch.bin', type=str, metavar='FILENAME', help='checkpoint to evaluate (file name)')
    #parser.add_argument('-e', '--evaluate', default='/home/hrai/codes/MotionBERT/checkpoint/pose3d/MB_ft_h36m/best_epoch.bin', type=str, metavar='FILENAME', help='checkpoint to evaluate (file name)')
    parser.add_argument('-e', '--evaluate', default='/home/hrai/codes/MotionBERT/checkpoint/pose3d/FT_MB_ft_h36m_MB_ft_aihub_30_pious-glitter-22/epoch_59.bin', type=str, metavar='FILENAME', help='checkpoint to evaluate (file name)')
    #parser.add_argument('-ms', '--selection', default='', type=str, metavar='FILENAME', help='checkpoint to finetune (file name)')
    parser.add_argument('-sd', '--seed', default=0, type=int, help='random seed')
    opts = parser.parse_args([])
    return opts

def set_random_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    
os.chdir('..')
opts = parse_args()
set_random_seed(opts.seed)
args = get_config(opts.config)

In [31]:
model_backbone = load_backbone(args)
if torch.cuda.is_available():
    model_backbone = nn.DataParallel(model_backbone)
    model_backbone = model_backbone.cuda()

print('Loading checkpoint', opts.evaluate)
checkpoint = torch.load(opts.evaluate, map_location=lambda storage, loc: storage)
model_backbone.load_state_dict(checkpoint['model_pos'], strict=True)
model_pos = model_backbone
model_pos.eval()

Loading checkpoint /home/hrai/codes/MotionBERT/checkpoint/pose3d/FT_MB_ft_h36m_MB_ft_aihub_30_pious-glitter-22/epoch_59.bin


DataParallel(
  (module): DSTformer(
    (joints_embed): Linear(in_features=3, out_features=512, bias=True)
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks_st): ModuleList(
      (0): Block(
        (norm1_s): LayerNorm((512,), eps=1e-06, elementwise_affine=True)
        (norm1_t): LayerNorm((512,), eps=1e-06, elementwise_affine=True)
        (attn_s): Attention(
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=512, out_features=512, bias=True)
          (qkv): Linear(in_features=512, out_features=1536, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (attn_t): Attention(
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=512, out_features=512, bias=True)
          (qkv): Linear(in_features=512, out_features=1536, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2_s): LayerNorm((512,), eps=1e-06,

In [14]:
# 예제 CLIP 파일
os.chdir('/home/hrai/codes/MotionBERT')
# with open('data/motion3d/MB3D_f243s81/AIHUB_243/train/00000025.pkl', 'rb') as f:
# 	test = pickle.load(f)
# frame_num = 150

# with open('data/motion3d/MB3D_f243s81/AIHUB_SPORTS_243/train/00000050.pkl', 'rb') as f:
# 	test = pickle.load(f)
# frame_num = 100

with open('data/motion3d/MB3D_f243s81/AIHUB_30_cam4_243/train/00000000.pkl', 'rb') as f:
	test = pickle.load(f)
frame_num = 150
batch_input = torch.tensor(test['data_input'][frame_num].reshape(1, 1, 17, 3))

In [27]:
test['data_info'][frame_num]

{'action_id': '30',
 'actor': 'M160B',
 'cam_num': '4',
 'frame_num': '101',
 'image_id': '30_M160B_4_101.jpg'}

In [32]:
result = model_pos(batch_input.float())
result_array = result[0][0].cpu().detach().numpy()

In [17]:
# 2d input
aihub_2d = copy.deepcopy(test['data_input'][frame_num])
# 2d skeleton
pos = copy.deepcopy(aihub_2d)
points_aihub_2d = {}
for i, p in enumerate(pos):
    p = np.append(p, 0.0)
    #print(i, h36m_keypoints[i], p)
    points_aihub_2d[i] = p
pos_vector = pos.reshape(-1) # (51,)
total_min, total_max = pos_vector.min(), pos_vector.max()
print("total min: ", total_min,
      "total max: ", total_max)

interact(draw_skeleton_2d(points_aihub_2d, h36m_connections))

total min:  -0.21606477101643878 total max:  0.27968311309814453


In [18]:
# gt
aihub_3d = copy.deepcopy(test['data_label'][frame_num])
pos = copy.deepcopy(aihub_3d)
points_aihub_3d = {}
for i, p in enumerate(pos):
    #print(i, h36m_keypoints[i], p)
    points_aihub_3d[i] = np.array([p[0], -p[2], -p[1]]) #np.array([p[0], p[1], p[2]])  #np.array([p[0], -p[2], p[1]])
pos_vector = pos.reshape(-1) # (51,)
total_min, total_max = pos_vector.min(), pos_vector.max()
print("total min: ", total_min,
      "total max: ", total_max)

interact(draw_skeleton(points_aihub_3d, h36m_connections))

total min:  -0.7334999999999999 total max:  0.15136666666666668


In [33]:
# result
aihub_3d = copy.deepcopy(result_array)
pos = copy.deepcopy(aihub_3d)
points_aihub_3d = {}
for i, p in enumerate(pos):
    #print(i, h36m_keypoints[i], p)
    points_aihub_3d[i] = np.array([p[0], -p[2], -p[1]]) #np.array([p[0], p[1], p[2]])  #np.array([p[0], -p[2], p[1]])
pos_vector = pos.reshape(-1) # (51,)
total_min, total_max = pos_vector.min(), pos_vector.max()
print("total min: ", total_min,
      "total max: ", total_max)

interact(draw_skeleton(points_aihub_3d, h36m_connections))

total min:  -0.40472943 total max:  0.37494442


### Save video

In [84]:
!pip install -U kaleido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 7.5 MB/s eta 0:00:00:00:0100:01


In [1]:
import plotly.express as px
import pandas as pd
import numpy as np
import io
import PIL

r = np.random.RandomState(42)

# sample data
df = pd.DataFrame(
    {
        "step": np.repeat(np.arange(0, 8), 10),
        "x": np.tile(np.linspace(0, 9, 10), 8),
        "y": r.uniform(0, 5, 80),
    }
)

# smaple plotly animated figure
fig = px.bar(df, x="x", y="y", animation_frame="step")

# generate images for each step in animation
frames = []
for s, fr in enumerate(fig.frames):
    # set main traces to appropriate traces within plotly frame
    fig.update(data=fr.data)
    # move slider to correct place
    fig.layout.sliders[0].update(active=s)
    # generate image of current state
    frames.append(PIL.Image.open(io.BytesIO(fig.to_image(format="png"))))
    
# create animated GIF
frames[0].save(
        "test.gif",
        save_all=True,
        append_images=frames[1:],
        optimize=True,
        duration=500,
        loop=0,
    )

In [30]:
# 2d input video
import plotly.express as px
import pandas as pd
import numpy as np
import io
import PIL


frames = []
connections = h36m_connections

for frame_num in range(243):

    aihub_2d = copy.deepcopy(test['data_input'][frame_num])

    # 2d skeleton
    pos = copy.deepcopy(aihub_2d)
    points = {}
    for i, p in enumerate(pos):
        p = np.append(p, 0.0)
        #print(i, h36m_keypoints[i], p)
        points[i] = p

    

    fig = go.Figure()

    # Add points
    for point in points.values():
        fig.add_trace(go.Scatter(x=[point[0]], y=[point[1]],
                                    mode='markers', marker=dict(size=2, color='blue')))

    # Add connections
    for connection in connections:
        p1, p2 = points[connection[0]], points[connection[1]]
        fig.add_trace(go.Scatter(x=[p1[0], p2[0]], y=[p1[1], p2[1]],
                                    mode='lines', line=dict(color='black')))

    # https://wikidocs.net/185955 참고
    fig.update_xaxes(range=[-1, 1])
    fig.update_yaxes(range=[1, -1])
    fig.update_layout(width=700,height=700)
    fig.update_layout(template="plotly_white")

    frames.append(PIL.Image.open(io.BytesIO(fig.to_image(format="png"))))
    
# create animated GIF
frames[0].save(
        "test.gif",
        save_all=True,
        append_images=frames[1:],
        optimize=True,
        duration=33,
        loop=0,
    )

In [32]:
# 3d result video
import plotly.express as px
import pandas as pd
import numpy as np
import io
import PIL


frames = []
connections = h36m_connections

for frame_num in range(243):

    batch_input = torch.tensor(test['data_input'][frame_num].reshape(1, 1, 17, 3))
    result = model_pos(batch_input.float())
    result_array = result[0][0].cpu().detach().numpy()

    pos = copy.deepcopy(result_array)
    points = {}
    for i, p in enumerate(pos):
        #print(i, h36m_keypoints[i], p)
        points[i] = np.array([p[0], -p[2], -p[1]]) #np.array([p[0], p[1], p[2]])  #np.array([p[0], -p[2], p[1]])
    

    fig = go.Figure()

    # Add points
    for point in points.values():
        fig.add_trace(go.Scatter(x=[point[0]], y=[point[1]],
                                    mode='markers', marker=dict(size=2, color='blue')))

    # Add connections
    for connection in connections:
        p1, p2 = points[connection[0]], points[connection[1]]
        fig.add_trace(go.Scatter(x=[p1[0], p2[0]], y=[p1[1], p2[1]],
                                    mode='lines', line=dict(color='black')))

    elevation=0 
    azimuth=0
    xaxis=dict(range=[-1, 1])
    yaxis=dict(range=[-1, 1])
    zaxis=dict(range=[-1, 1])

    # Set layout
    fig.update_layout(scene=dict(xaxis=xaxis, yaxis=yaxis, zaxis=zaxis,
                                 aspectratio=dict(x=1, y=1, z=1),
                                 camera=dict(eye=dict(x=np.cos(np.pi * azimuth / 180) * np.cos(np.pi * elevation / 180),
                                                      y=np.sin(np.pi * azimuth / 180) * np.cos(np.pi * elevation / 180),
                                                      z=np.sin(np.pi * elevation / 180)))),
                      width=700, height=700, autosize=False,
                      scene_camera_eye=dict(x=1, y=1, z=1))

    frames.append(PIL.Image.open(io.BytesIO(fig.to_image(format="png"))))
    
# create animated GIF
frames[0].save(
        "3d_result.gif",
        save_all=True,
        append_images=frames[1:],
        optimize=True,
        duration=33,
        loop=0,
    )